In [ ]:
# This produces the dataframe for WR

In [ ]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates
# best to run this on a day other than Tues.

In [ ]:
## REQUIRED ACTIONS ##
# modify the season start date 

In [1]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re

In [5]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [7]:
# Function to get the current NFL week and year
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Reset this date at the start of the NFL season
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Set the current NFL year and week
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [21]:
print(current_week)

18


In [33]:
# columns for season data
season_columns = [
    'season', 'season_type', 'week'
]

In [35]:
print(season_columns)

['season', 'season_type', 'week']


In [56]:
# columns for athlete id data
# Pull the ids from the nfl-data-py library
# nfl.import_ids(columns, ids)
# nfl.import_weekly_data(years, columns, downcast)
athlete_columns = [
    'player_id', 
    'gsis_id',
    'mfl_id',	
    'sportradar_id',
    'fantasypros_id',
    'pff_id',	
    'sleeper_id',
    'nfl_id',	
    'espn_id',
    'yahoo_id',
    'fleaflicker_id',
    'cbs_id',
    'pfr_id',
    'cfbref_id',
    'rotowire_id',	
    'rotoworld_id',
    'ktc_id	stats_id',	
    'stats_global_id',
    'fantasy_data_id',
    'swish_id',
    'player_name', 
    'position', 
    'position_group', 
    'recent_team'
    ]

In [58]:
print(athlete_columns)

['player_id', 'mfl_id', 'sportradar_id', 'fantasypros_id', 'gsis_id', 'pff_id', 'sleeper_id', 'nfl_id', 'espn_id', 'yahoo_id', 'fleaflicker_id', 'cbs_id', 'pfr_id', 'cfbref_id', 'rotowire_id', 'rotoworld_id', 'ktc_id\tstats_id', 'stats_global_id', 'fantasy_data_id', 'swish_id', 'player_name', 'position', 'position_group', 'recent_team']


In [ ]:
### NEXT TASK ###
# merge the weekly WR data and the id data
## Note: merge based on'player_id' and gsis_id' since they match!! 

In [66]:
# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

In [68]:
print(wr_columns)

['receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr']


In [71]:
# Import data for all the specified years
years = list(range(2017, current_year + 1))
nfl_data_all_years = nfl.import_weekly_data(
    years=years,
    columns=wr_columns
)

HTTPError: HTTP Error 404: Not Found